# IASI functions

In [ ]:
def IASI_version(component_nom, year):

    """ Get version of IASI dataset for each component nomenclature

         Args:
            component_nom (str): Component chemical nomenclature
            year (str): Year of dataset

        Returns:
            version (str): IASI dataset version
    """

    # https://iasi.aeris-data.fr/CO_IASI_C_L3_data/
    if component_nom == 'CO':
        version = 'V6.5.0'

    # https://iasi.aeris-data.fr/O3_IASI_C_L3_data/
    elif component_nom == 'O3':
        version = 'V6.5.1'

    # https://iasi.aeris-data.fr/HCOOH_IASI_C_L3_data/
    elif component_nom == 'HCOOH':
        version = 'V1.0.0'

    return version

In [ ]:
def IASI_download(component_nom, date):

    """ Download the IASI dataset with curl

         Args:
            component_nom (str): Component chemical nomenclature
            date (str): Query month as date

        Returns:
            product_name (str): Product name of IASI product
    """

    cn = component_nom.lower()
    year = date.split('-')[0]
    month = date.split('-')[1]
    version = IASI_version(component_nom, year)
    product_name = ''.join(['IASI_METOPC_L3_', component_nom, '_COLUMN_', year, month, '_ULB-LATMOS_', version, '.nc'])
    
    print(f'Downloading {product_name}...')
    !curl --insecure https://cds-espri.ipsl.fr/iasicl3/iasi_$cn/$version/$year/$product_name --output data/iasi/$component_nom/$product_name
    
    return product_name

In [ ]:
def IASI_read(product_names, component_nom):

    """ Read the IASI dataset as xarray dataset object and assign time

         Args:
            product_names (arr): Product names of IASI products
            component_nom (str): Component chemical nomenclature

        Returns:
            sensor_ds (xarray): IASI dataset in xarray format
            sensor_type (str): Sensor type ('NRT')
    """

    sensor_ds_all = []

    for product_name in product_names:

        sensor_ds = xr.open_dataset('data/iasi/' + component_nom + '/' + product_name)

        # Add datetime (e.g.: for June 2021, 2021-06-01)
        year = int(product_name.split('COLUMN_')[1].split('_ULB')[0][0:4])
        month = int(product_name.split('COLUMN_')[1].split('_ULB')[0][4:6])
        time_str = datetime.datetime(year, month, 1)
        sensor_ds = sensor_ds.assign_coords({'time': time_str}).expand_dims(dim = ['time'])

        sensor_ds_all.append(sensor_ds)
    
        # Merge if there is more than one product
        if len(product_names) >= 2:

            sensor_ds = xr.merge(sensor_ds_all)

    sensor_type = 'L3'

    return sensor_ds, sensor_type